<a href="https://colab.research.google.com/github/vikramsharitas/PS-1-Project/blob/master/Copy_of_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
import tensorflow as tf

import os
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1px3LZ7Twk5YuPio48YpG4Iy3Li9ag0bC'

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('telemetry.csv')  
df = pd.read_csv('telemetry.csv')

In [0]:
hoursto = np.zeros(8761)
hourssince = np.zeros(8761)
l = len(df['yval'])
temp1 = None
temp2 = None
for i in range(8760, -1, -1):
  if df['yval'][i] == 1:
    temp1 = 1
    hoursto[i] = 0
    continue
  if temp1 == 1:
    hoursto[i] = hoursto[i+1] + 1
  else:
    hoursto[i] = None
  if df['yval'][8760-i] == 1:
    temp2 = 1
    hourssince[8760-i] = 0
    continue
  if temp2 == 1:
    hourssince[8760-i] = hourssince[8760-i-1] + 1
  else:
    hourssince[8760-i] = None
df['hoursto'] = hoursto
df['hourssince'] = hourssince

In [0]:
df.head(8761)

In [0]:
features_considered = ['volt', 'rotate', 'pressure', 'vibration', 'hourssince', 'hoursto']
features = df[features_considered]
index = ['datetime', 'machineID']
features.index = df['datetime']
features.head()

In [0]:
# features.plot(subplots=True)

In [0]:
TRAIN_SPLIT = 7157
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
data_var = dataset[:TRAIN_SPLIT].var(axis=0)

In [0]:
print(data_mean)
print(data_std)
print(data_var)

In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size + 1

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
def create_time_steps(length):
  return list(range(-length, 0))

In [0]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [0]:
past_history = 744
future_target = 744
STEP = 1
print(len(dataset))

In [0]:
x_train_multi, y_train_multi = multivariate_data(dataset[:, :5], dataset[:, 5], 0,
                                                 TRAIN_SPLIT-future_target+1, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset[:, :5], dataset[:, 5],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

In [0]:
BUFFER_SIZE = 1000
BATCH_SIZE = 9
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE)

In [0]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 5]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In [0]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(256, return_sequences=True, input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(256))
multi_step_model.add(tf.keras.layers.Flatten())
# multi_step_model.add(tf.keras.layers.Conv1D(filters=200, kernel_size=5, strides=1, padding="causal", activation="relu"))
multi_step_model.add(tf.keras.layers.Dense(180))
multi_step_model.add(tf.keras.layers.Dense(future_target))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')
print(multi_step_model.summary())

In [0]:
for x, y in val_data_multi.take(1):
  print (multi_step_model.predict(x).shape)

In [0]:
EVALUATION_INTERVAL = 300
EPOCHS = 5

In [0]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          validation_data=val_data_multi)

In [0]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

In [0]:
for x, y in val_data_multi.take(1):
  y0 = multi_step_model.predict(x)[0]
  print(y0)
  print(y[0])
  multi_step_plot(y[0,:], y[0,:], y0)